In [22]:
# Harrow Hassidim Lloyd algorithm(HHL)

#import necessary frameworks
import cirq 
import numpy as np 
import sympy

# define the matrix A and vector b
A = np.array([[0,1],[1,0]])
b = np.array([1,0])

# create qubits
ancilla = cirq.NamedQubit('ancilla')
register = [ cirq.NamedQubit(f"register_{i}") for i in range(2) ]
memory = [ cirq.NamedQubit(f"memory_{i}") for i in range(2) ]

# define circuit 
circuit = cirq.Circuit()

# Quantum phase estimation
circuit.append(cirq.H.on_each(register))
circuit.append(cirq.ControlledGate(cirq.MatrixGate(A)).on(register[0],memory[0]))
circuit.append(cirq.qft(*register))

# Controlled rotations
theta = sympy.Symbol('theta')
circuit.append(cirq.ry(theta).controlled().on(register[0],ancilla))

# Inverse QPE
circuit.append(cirq.qft(*register,inverse=True))

# Measure the ancilla 
circuit.append(cirq.measure(ancilla, key='result'))

print(circuit)

# simulate
sim = cirq.Simulator()
r = sim.run(circuit, param_resolver = {theta : np.pi / 4},repetitions=1000)
print(r.histogram(key='result'))

ancilla: ────────────────────────Ry(theta)───M('result')───
                                 │
                   ┌   ┐         │
memory_0: ─────────│0 1│─────────┼─────────────────────────
                   │1 0│         │
                   └   ┘         │
                   │             │
register_0: ───H───@───────qft───@───────────qft^-1────────
                           │                 │
register_1: ───H───────────#2────────────────#2────────────
Counter({0: 964, 1: 36})
